In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import os
from PIL import Image
import array as arr
from sklearn.preprocessing import MinMaxScaler
import skimage.feature
from skimage.feature import greycomatrix
from skimage.filters import rank, try_all_threshold
from skimage.morphology import disk

# Read Hourly Dataset

In [2]:
HomeLi = [pd.read_csv("HrData1.csv"),
          pd.read_csv("HrData2.csv"),
          pd.read_csv("HrData3.csv"),
          pd.read_csv("HrData4.csv"),
          pd.read_csv("HrData5.csv"),
          pd.read_csv("HrData6.csv"),
          pd.read_csv("HrData7.csv")]

# Drop unwanted Columns

In [3]:
for i in range(len(HomeLi)):
    HomeLi[i].drop(['total','Unnamed: 0'],axis=1,inplace=True)

# Remove errors & missing values

In [4]:
err = pd.DataFrame({'patient': [], 'column': [], 'row': []})

for i in range(len(HomeLi)):
    for j in range(1,12):
        for k in range(len(HomeLi[i])):
            if ((HomeLi[i].iloc[k,j])<0) or ((HomeLi[i].iloc[k,j])==None):
                err.loc[len(err.index)] = [i, j, k]

# Seasonal normalisation

In [24]:
#Change i from 0 to 6 for patient datasets


summer = []
autumn = []
winter = []
spring = []

for i in range(4,5):
    for j in range(0,len(HomeLi[i]),24):
        month = HomeLi[i].iloc[j,0]
        if month[5:7] == '08' or month[5:7] == '06' or month[5:7] == '07':
            summer.append(HomeLi[i][j:j+24])
        if month[5:7] == '09' or month[5:7] == '10' or month[5:7] == '11':
            autumn.append(HomeLi[i][j:j+24])
        if month[5:7] == '12' or month[5:7] == '01' or month[5:7] == '02':
            winter.append(HomeLi[i][j:j+24])
        if month[5:7] == '03' or month[5:7] == '04' or month[5:7] == '05':
            spring.append(HomeLi[i][j:j+24])
            
            
CSummer = pd.DataFrame()
for s in range(0,len(summer)):
    CSummer = CSummer.append(summer[s], ignore_index=True)
CAutumn = pd.DataFrame()
for s in range(0,len(autumn)):
    CAutumn = CAutumn.append(autumn[s], ignore_index=True)
Cwinter = pd.DataFrame()
for s in range(0,len(winter)):
    Cwinter = Cwinter.append(winter[s], ignore_index=True)
Cspring = pd.DataFrame()
for s in range(0,len(spring)):
    Cspring = Cspring.append(spring[s], ignore_index=True)
    

TimeSummer = CSummer.iloc[:,0:2]
TimeAutumn = CAutumn.iloc[:,0:2]
TimeSpring = Cspring.iloc[:,0:2]
TimeWinter = Cwinter.iloc[:,0:2]


CSummer.drop(['date','hour'],axis=1,inplace=True)
CAutumn.drop(['date','hour'],axis=1,inplace=True)
Cwinter.drop(['date','hour'],axis=1,inplace=True)
Cspring.drop(['date','hour'],axis=1,inplace=True)


Smin_value, Smax_value = CSummer.min().min(), CSummer.max().max()
Summerscaled1 = (CSummer - Smin_value) * 1 / (Smax_value - Smin_value)

Amin_value, Amax_value = CAutumn.min().min(), CAutumn.max().max()
Autumnscaled1 = (CAutumn - Amin_value) * 1 / (Amax_value - Amin_value)

Wmin_value, Wmax_value = Cwinter.min().min(), Cwinter.max().max()
Winterscaled1 = (Cwinter - Wmin_value) * 1 / (Wmax_value - Wmin_value)

Spmin_value, Spmax_value = Cspring.min().min(), Cspring.max().max()
Springscaled1 = (Cspring - Spmin_value) * 1 / (Spmax_value - Spmin_value)


summerNew = pd.concat([TimeSummer, Summerscaled1], axis=1)
autumnNew = pd.concat([TimeAutumn, Autumnscaled1], axis=1)
winterNew = pd.concat([TimeWinter, Winterscaled1], axis=1)
springNew = pd.concat([TimeSpring, Springscaled1], axis=1)


Patient = pd.DataFrame()
Patient = Patient.append(summerNew, ignore_index=True)
Patient = Patient.append(autumnNew, ignore_index=True)
Patient = Patient.append(winterNew, ignore_index=True)
Patient = Patient.append(springNew, ignore_index=True)

C:\Users\Anisha Jubitha\AppData\Local\Temp\ipykernel_14580\61845585.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CSummer = CSummer.append(summer[s], ignore_index=True)
C:\Users\Anisha Jubitha\AppData\Local\Temp\ipykernel_14580\61845585.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CAutumn = CAutumn.append(autumn[s], ignore_index=True)
C:\Users\Anisha Jubitha\AppData\Local\Temp\ipykernel_14580\61845585.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Cwinter = Cwinter.append(winter[s], ignore_index=True)
C:\Users\Anisha Jubitha\AppData\Local\Temp\ipykernel_14580\61845585.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

# Generate Heatmaps

In [ ]:
for j in range(0,len(Patient),24):
    df = pd.DataFrame(Patient[j:j+24])
        
    df = pd.pivot_table(Patient[j:j+24],index = 'hour') 
    mms = MinMaxScaler()
    df_norm = mms.fit_transform(df)
        
    normalised_hmap = sb.heatmap(df_norm, linewidths=.5, cmap='viridis', cbar=False, xticklabels=['backdoor', 'bathroom', 'lounge', 'frontdoor', 'bedroom', 'kettle', 'hallway',  'kitchen', 'fridgedoor', 'backdoor', 'oven'])
    fig = normalised_hmap.get_figure()
    fig.savefig("HM4/hour"+str(j)+".png",bbox_inches ="tight")